# Capítulo extra — EM para repartir contribuciones de MMM a campañas/creatividades (MTA)

## 0) Qué es observado y qué es latente

### Observado (sale del MMM)

Tienes para cada periodo (y opcionalmente geo):

* ($t \in {1,\dots,T}$) (semana/día)
* ($g \in {1,\dots,G}$) (geo) opcional

Y del MMM obtienes contribuciones por canal/medio:

* ($C_{m,t}$) o ($C_{m,t,g}$): contribución del **medio/canal** (m) en ese corte.

> Nota: esto puede ser “incremental contribution” del MMM o “attributed contribution” según el modelo. EM solo lo toma como “total a repartir”.

### Objetivo

Repartir esa contribución de cada medio (m) hacia unidades inferiores:

* campañas (k), creatividades, adsets, publishers… (lo que definas)

### Latente

* ($z_{m,k,t}$) (o ($z_{m,k,t,g}$)): contribución del medio (m) en (t) asignada a campaña/crea (k)

Restricción dura:

$$
\sum_{k\in \mathcal{K}*m} z*{m,k,t} = C_{m,t}
\quad (\text{o } \sum_k z_{m,k,t,g}=C_{m,t,g})
$$

---



## 1) Modelo generativo base (análogamente al HCP–Brick)

La analogía es directa:

| HCP–Brick            | MMM→MTA                                                   |
| -------------------- | --------------------------------------------------------- |
| Brick                | (medio (m), tiempo (t), geo (g))                          |
| HCP                  | campaña/creatividad (k) dentro del medio                  |
| ventas (y_b)         | contribución (C_{m,t,g})                                  |
| exposición (w_{h,b}) | señales a nivel campaña: spend, impressions, clicks, etc. |

### Exposición / evidencia por campaña

Define features o exposición para cada campaña:

* (w_{m,k,t,g}) = spend (o impresiones) de la campaña (k) del medio (m) en (t,g)
* y opcionalmente features (X_{m,k}) (tipo campaña, objetivo, formato…)

### Productividad latente por campaña dentro del medio

Caso simple (sin covariables aún):

* (\lambda_{m,k} > 0) (tasa/productividad relativa de campaña (k) dentro de medio (m))

Modelo Poisson (para “unidades de contribución”):
[
z_{m,k,t,g} \sim \text{Poisson}(\lambda_{m,k}, w_{m,k,t,g})
]
y observamos:
[
C_{m,t,g}=\sum_k z_{m,k,t,g}
]

Si (C) es continua (euros), puedes usar Gamma (más abajo).

---

## 2) EM explícito en este contexto (solución)

### E-step: repartir (C_{m,t,g}) dentro de cada (m,t,g)

Define:
[
\mu_{m,k,t,g}^{(r)} = \lambda_{m,k}^{(r)}, w_{m,k,t,g}
]
[
p_{m,k,t,g}^{(r)}=
\frac{\mu_{m,k,t,g}^{(r)}}{\sum_{k'}\mu_{m,k',t,g}^{(r)}}
]

Entonces:
[
\boxed{
\mathbb{E}[z_{m,k,t,g}\mid C_{m,t,g},\lambda^{(r)}] = C_{m,t,g} \cdot p_{m,k,t,g}^{(r)}
}
]

Interpretación:

* dentro de cada medio-tiempo-geo, la campaña se lleva más contribución si tiene:

  * más exposición (w) y/o
  * más productividad (\lambda).

### M-step: actualizar (\lambda_{m,k})

Acumulas:
[
\tilde z_{m,k}=\sum_{t,g}\mathbb{E}[z_{m,k,t,g}]
\quad,\quad
\tilde w_{m,k}=\sum_{t,g} w_{m,k,t,g}
]

MLE tipo Poisson:
[
\boxed{
\lambda_{m,k}^{(r+1)} = \frac{\tilde z_{m,k}}{\tilde w_{m,k}}
}
]

Eso ya es EM completo (sin covariables).

---

## 3) Versión “realista”: covariables + regularización

Como antes:
[
\lambda_{m,k}=\exp(X_{m,k}\beta_m)
]

* (\beta_m) puede ser por medio, o compartido

M-step se vuelve un **Poisson GLM con offset**:

* response: (\tilde z_{m,k})
* offset: (\log \tilde w_{m,k})
* regularización ridge/lasso para estabilidad

---

## 4) Caso euros (contribuciones continuas)

Si (C_{m,t,g}) es continua (moneda), Gamma suele funcionar bien:

[
z_{m,k,t,g} \sim \text{Gamma}(k,\ \theta_{m,k} w_{m,k,t,g})
]
con (\theta_{m,k}=\exp(X\beta))

En muchas parametrizaciones razonables, el reparto del E-step queda (muy similar):
[
\mathbb{E}[z_{m,k,t,g}\mid C_{m,t,g}]
\approx
C_{m,t,g}\cdot
\frac{\theta_{m,k} w_{m,k,t,g}}{\sum_{k'}\theta_{m,k'} w_{m,k',t,g}}
]
y el M-step es Gamma-GLM (log link) + regularización.

---

## 5) Qué estás aprendiendo realmente (y límites)

Este EM aprende **productividad relativa intra-medio** de campañas/creas **consistente con**:

* la serie temporal (y geo) de contribución total del MMM
* el patrón de exposición de campañas (spend/imp/clicks)

**No** está recuperando “verdad causal por campaña” si:

* el MMM no está bien identificado por canal
* hay campañas altamente colineales dentro del medio
* falta señal (w) (o está muy ruidosa)

Pero como “desagregador coherente” del MMM, funciona muy bien.

---

## 6) Lo de “granularidad horaria” en MTA (para más adelante)

Tu comentario es clave: MMM suele estar en semanal/día; MTA a veces se quiere a hora/minuto.

Hay 3 rutas típicas:

1. **Downscaling con modelo de intensidad**
   Aprendes un “perfil intradía” (q(h \mid k,g)) con logs de delivery (imp/click/spend horario) y repartes:
   [
   z_{m,k,t,g,h} = z_{m,k,t,g}\cdot q(h\mid k,g)
   ]
   Esto es determinista o con EM a dos niveles.

2. **Estado-espacio / smoothing temporal**
   Tratas (\lambda_{m,k,t}) como proceso (random walk) y haces inferencia (Kalman/VI).
   Útil si quieres que productividades varíen por hora.

3. **Modelo unificado a alta frecuencia**
   Solo si tienes outcomes y gasto a esa frecuencia (caro y con mucho ruido).

---

# Ejercicio propuesto (con solución)

**Ejercicio:** Implementa EM para un solo medio (m) con:

* (C_t) semanal
* campañas (k=1..K)
* exposición (w_{k,t}) = spend
  y estima (\lambda_k) + (z_{k,t}).

**Solución:** las ecuaciones de arriba (E-step/M-step) y luego extender a covariables con GLM.

---

## ¿Quieres que lo aterricemos ya en código?

Puedo darte un notebook muy parecido al del HCP–Brick pero con:

* input: `C_{m,t,g}` (del MMM) + `w_{m,k,t,g}` (spend/imp/click)
* output: `zhat_{m,k,t,g}` y productividades por campaña

Y con dos modos:

* “sin covariables” (updates cerrados)
* “con covariables” (M-step con IRLS + ridge)

Si lo hacemos, asumiré por defecto:

* contribución (C) continua pero **no negativa**
* exposición = spend (>=0), con smoothing cuando hay ceros

Dime solo si quieres que empecemos por:

1. **1 medio, sin geo** (más simple), o
2. **1 medio, con geo**, o
3. **multi-medio** (mismo framework, un loop por medio).


Genial: si quieres usar **impresiones/GRPs** y además respetar **adstock + saturación** (como en MMM), eso cambia el “qué significa exposición” en el EM, pero el framework encaja muy bien.

Voy por tus 2 preguntas.

---

## 1) Si el MMM es incremental (causal), ¿el reparto MTA también lo es?

**Depende de qué entiendas por “causal” a nivel campaña**, pero la respuesta operativa es:

* **El total que repartes (por canal/medio y por (t), (g)) sí es incremental/causal** *si tu MMM lo es*.
* **El reparto dentro del canal (campañas/creas)** **no es automáticamente causal** a menos que tengas **identificación causal intra-canal**.

### Por qué

El MMM “incremental” te identifica algo como:
[
C_{m,t,g}^{inc} = f(\text{inputs del canal }m) - f(\text{contrafactual sin canal }m)
]
Eso es causal a nivel canal (bajo supuestos).

Pero cuando lo repartes entre campañas, normalmente estás usando:

* señales de delivery (impresiones/GRPs, targeting, timing)
* y un modelo de “propensión” / “intensidad”
  para decidir shares.

Eso da una atribución **coherente** con el incremental del canal, pero **no demuestra** que “esa campaña” sea incremental vs otra, porque dentro del canal puede haber:

* colinealidad fuerte (campañas corren a la vez)
* selección/endogeneidad (las campañas se activan cuando se espera demanda)
* distinta mezcla de audiencias

### Cómo sí hacerlo “más causal” intra-canal

Necesitarías señal de identificación adicional, por ejemplo:

* experimentos / geo-lift por campaña
* cambios exógenos (holdouts, rotaciones creativas, shocks de supply)
* instrumentos o reglas de asignación
* o un modelo jerárquico que incorpore “priors causales” basados en pruebas

📌 Conclusión clara:

> **MTA por EM puede conservar el “incremental total” del MMM**, pero el **split intra-canal es principalmente model-based attribution**, no causal garantizado.

---

## 2) Cómo insertar adstock + saturación en la solución EM (con impresiones/GRPs)

La forma limpia es: **no usar exposición cruda (w)**, sino la **exposición efectiva** tal y como MMM la usa para generar contribución.

### En MMM típico

Para una subentidad (k) (campaña/crea) del canal (m), tienes una señal “raw”:

* (x_{m,k,t,g}): impresiones / GRPs / TRPs (no spend)

MMM aplica transformaciones:

1. **Adstock** (carryover):
   [
   a_{m,k,t,g} = \text{Adstock}(x_{m,k,\cdot,g}; \theta_m)
   ]
   (p. ej. geométrico con decay (\theta_m), o Weibull)

2. **Saturación** (rendimientos decrecientes):
   [
   s_{m,k,t,g} = \text{Sat}(a_{m,k,t,g}; \phi_m)
   ]
   (p. ej. Hill / logistic)

Y la contribución incremental del canal suele ser algo como:
[
C_{m,t,g}^{inc} \approx \beta_m \sum_k s_{m,k,t,g}
]
(si el MMM no tiene campañas explícitas, esto es una descomposición plausible)

---

### Cómo meter esto en EM (dos opciones)

#### ✅ Opción A (recomendada): EM reparte usando **exposición efectiva MMM-consistente**

Define la exposición del EM como:
[
w_{m,k,t,g} := s_{m,k,t,g}
]
(es decir, **ya adstockeada y saturada**)

Entonces el E-step (shares) queda:
[
\hat z_{m,k,t,g} = C_{m,t,g}^{inc}\cdot
\frac{\lambda_{m,k} \cdot s_{m,k,t,g}}
{\sum_{k'} \lambda_{m,k'} \cdot s_{m,k',t,g}}
]

Ventajas:

* Mantienes exactamente la lógica MMM (carryover + diminishing returns)
* Evitas atribuir más a campañas solo por impresiones brutas si están saturadas
* Muy estable y fácil de explicar

Qué se aprende:

* (\lambda_{m,k}) = “eficiencia relativa” intra-canal (sobre señal efectiva)

Dónde pones parámetros de adstock/sat:

* Los tomas del MMM (fijos) o los fijas por canal.

#### Opción B: EM también aprende algunos parámetros de transformación

Puedes extender parámetros (\theta_m,\phi_m) al set de parámetros y alternar:

* E-step: imputar (z)
* M-step: actualizar (\beta) y quizás (\theta,\phi)

Pero aquí hay un problema práctico:

* adstock/saturación + shares suelen estar **muy no convexos**
* identificabilidad débil (especialmente intra-canal)
* entrenamiento más frágil (necesitarías constraints/priors fuertes)

📌 Para empezar y para producción, casi siempre:

> **usar (\theta_m,\phi_m) del MMM (fijos) y aprender solo el reparto intra-canal**.

---

### ¿Y si el MMM está a nivel canal y no tienes series por campaña?

Entonces haces “top-down”:

* Construyes (s_{m,k,t,g}) a partir de delivery de campaña (imps/GRPs) usando los mismos (\theta_m,\phi_m)
* Y repartes (C_{m,t,g}^{inc}) con la fórmula anterior

Esto es exactamente lo que quieres.

---

## Recomendación final antes del notebook (sin pedirte datos extra)

Para el notebook te propongo este diseño:

* Inputs mock:

  * (C_{t}) o (C_{t,g}): contrib incremental por canal (observado)
  * (x_{k,t,g}): impresiones/GRPs por campaña (observado)
* Transformaciones:

  * adstock geométrico por canal
  * saturación Hill por canal
* EM:

  * E-step usando (w=s(x))
  * M-step estimando (\lambda_k) (cerrado) o (\lambda_k=\exp(X_k\beta)) (GLM)

Y añadimos sección “más adelante”:

* desagregación a horas via perfil intradía usando delivery horario + conservación de masa

---

## Checkpoint

Si estás OK con esto:

* **El total es causal (del MMM), el split intra-canal es attribution model-based**.
* Usamos **adstock+saturación para construir (w)** y EM reparte sobre esa señal efectiva.

Dime y genero el notebook del caso MMM→MTA con mock data (impresiones/GRPs + transforms).


# EM para desagregar contribuciones incrementales de MMM a MTA (campañas/creatividades)

Este notebook muestra un enfoque **top-down**:

- Un MMM produce contribuciones incrementales por **canal/medio** a nivel **tiempo** (y opcionalmente **geo**):  
  \( C_{m,t,g}^{inc} \)

- Queremos repartir \( C_{m,t,g}^{inc} \) a niveles inferiores (campañas/creas) \(k\) dentro del medio \(m\):  
  \( z_{m,k,t,g} \), con la restricción:  
  \( \sum_k z_{m,k,t,g} = C_{m,t,g}^{inc} \)

- Usamos como “exposición” una señal de delivery (impresiones/GRPs) **transformada como en MMM**:
  - **Adstock** (carryover)
  - **Saturación** (rendimientos decrecientes, p. ej. Hill)

El reparto (E-step) se hace proporcional a:
\[
\lambda_{m,k}\;\cdot\; s(\text{Adstock}(x_{m,k,t,g}))
\]
donde \(x\) son impresiones/GRPs y \(s(\cdot)\) es saturación.

> Nota causal: si el MMM es incremental/causal a nivel canal, el total \(C^{inc}\) lo es.  
> El split intra-canal es **atribución model-based**, no causal garantizado, salvo que haya identificación adicional intra-canal.


## 1) Utilidades: Adstock y Saturación (Hill)

In [1]:
import numpy as np
import pandas as pd

def adstock_geometric(x, theta):
    x = np.asarray(x, dtype=float)
    a = np.zeros_like(x)
    for t in range(len(x)):
        a[t] = x[t] + (theta * a[t-1] if t > 0 else 0.0)
    return a

def hill_saturation(x, alpha, gamma):
    x = np.asarray(x, dtype=float)
    x_pos = np.clip(x, 0.0, None)
    xa = np.power(x_pos + 1e-12, alpha)
    ga = np.power(gamma + 1e-12, alpha)
    return xa / (xa + ga)

def effective_exposure(x, theta, alpha, gamma):
    a = adstock_geometric(x, theta=theta)
    s = hill_saturation(a, alpha=alpha, gamma=gamma)
    return a, s

x = np.array([0, 10, 0, 0, 5, 0], dtype=float)
a, s = effective_exposure(x, theta=0.6, alpha=1.2, gamma=8.0)
pd.DataFrame({"x": x, "adstock": a, "sat": s})


,x,adstock,sat
0,0.0,0.000,3.283160e-16
1,10.0,10.000,5.665459e-01
2,0.0,6.000,4.145424e-01
3,0.0,3.600,2.772369e-01
4,5.0,7.160,4.667696e-01
5,0.0,4.296,3.216698e-01


## 2) Datos mock: canal (medio) -> campañas, con tiempo x geo

In [2]:
import numpy as np
import pandas as pd

rng = np.random.default_rng(123)

M = 2
K_per_m = 6
T = 52
G = 5

media_names = ["Search", "Social"]
geos = [f"G{i+1}" for i in range(G)]
weeks = np.arange(T)

theta_m = {"Search": 0.40, "Social": 0.65}
alpha_m = {"Search": 1.20, "Social": 1.40}
gamma_m = {"Search": 0.35, "Social": 0.25}

campaign_rows = []
for m in media_names:
    for k in range(K_per_m):
        campaign_rows.append({
            "medium": m,
            "campaign": f"{m}_C{k+1}",
            "creative_family": rng.choice(["A", "B", "C"]),
            "objective": rng.choice(["Awareness", "Consideration", "Conversion"], p=[0.3,0.4,0.3]),
        })
campaign_df = pd.DataFrame(campaign_rows)

geo_effect = rng.lognormal(mean=0.0, sigma=0.25, size=G)
time_season = 1.0 + 0.25*np.sin(2*np.pi*weeks/T) + 0.10*np.sin(4*np.pi*weeks/T)

x_rows = []
for _, row in campaign_df.iterrows():
    m = row["medium"]
    k = row["campaign"]
    base = rng.lognormal(mean=0.0, sigma=0.5)
    for g_idx, g in enumerate(geos):
        level = base * geo_effect[g_idx] * rng.lognormal(mean=0.0, sigma=0.15)
        x_t = level * time_season * rng.lognormal(mean=0.0, sigma=0.25, size=T)
        mask = rng.random(T) < 0.08
        x_t[mask] = 0.0
        for t in range(T):
            x_rows.append({"medium": m, "campaign": k, "geo": g, "t": int(t), "x_raw": float(x_t[t])})

x_df = pd.DataFrame(x_rows)

# Scale per medium to stabilize Hill gamma interpretation
x_df["x_scaled"] = x_df["x_raw"]
for m in media_names:
    med = np.median(x_df.loc[(x_df.medium==m) & (x_df.x_raw>0), "x_raw"])
    x_df.loc[x_df.medium==m, "x_scaled"] = x_df.loc[x_df.medium==m, "x_raw"] / (med + 1e-12)

x_df.head()


,medium,campaign,geo,t,x_raw,x_scaled
0,Search,Search_C1,G1,0,0.000000,0.000000
1,Search,Search_C1,G1,1,2.642698,1.932004
2,Search,Search_C1,G1,2,0.000000,0.000000
3,Search,Search_C1,G1,3,2.318439,1.694947
4,Search,Search_C1,G1,4,2.183659,1.596413


## 3) Ground truth: productividad intra-medio y generación de contribuciones incrementales del MMM

In [3]:
import numpy as np
import pandas as pd

lambda_true = {}
for m in media_names:
    lam = rng.lognormal(mean=0.0, sigma=0.6, size=K_per_m)
    lam = lam / lam.mean()
    camps = campaign_df.loc[campaign_df.medium==m, "campaign"].tolist()
    for i, k in enumerate(camps):
        lambda_true[(m, k)] = float(lam[i])

campaign_df["lambda_true"] = campaign_df.apply(lambda r: lambda_true[(r["medium"], r["campaign"])], axis=1)

# Effective exposure per (medium,campaign,geo) along time
s_rows = []
for (m,k,g), grp in x_df.groupby(["medium","campaign","geo"], sort=False):
    grp = grp.sort_values("t")
    x = grp["x_scaled"].to_numpy()
    a, s = effective_exposure(x, theta=theta_m[m], alpha=alpha_m[m], gamma=gamma_m[m])
    out = grp[["medium","campaign","geo","t"]].copy()
    out["adstock"] = a
    out["s_eff"] = s
    s_rows.append(out)
s_df = pd.concat(s_rows, ignore_index=True)

beta_m = {"Search": 120.0, "Social": 80.0}

sig = s_df.merge(campaign_df[["medium","campaign","lambda_true"]], on=["medium","campaign"], how="left")
sig["mu"] = sig["lambda_true"] * sig["s_eff"]

C_rows = []
for (m,t,g), grp in sig.groupby(["medium","t","geo"], sort=False):
    total_mu = grp["mu"].sum()
    mean = beta_m[m] * total_mu
    shape = 20.0
    scale = (mean / shape) if mean > 0 else 0.0
    C = float(rng.gamma(shape=shape, scale=scale)) if mean > 0 else 0.0
    C_rows.append({"medium": m, "t": int(t), "geo": g, "C_inc": C})
C_df = pd.DataFrame(C_rows)

sig2 = sig.merge(C_df, on=["medium","t","geo"], how="left")
sig2["share_true"] = sig2["mu"] / (sig2.groupby(["medium","t","geo"])["mu"].transform("sum") + 1e-12)
sig2["z_true"] = sig2["C_inc"] * sig2["share_true"]

sig2.head(), C_df.head(), campaign_df.head()


(   medium   campaign geo  t   adstock         s_eff  lambda_true  \
 0  Search  Search_C1  G1  0  0.000000  1.403196e-14     0.651796   
 1  Search  Search_C1  G1  1  1.932004  8.859533e-01     0.651796   
 2  Search  Search_C1  G1  2  0.772802  7.212172e-01     0.651796   
 3  Search  Search_C1  G1  3  2.004068  8.903188e-01     0.651796   
 4  Search  Search_C1  G1  4  2.398040  9.096500e-01     0.651796   
 
              mu       C_inc    share_true        z_true  
 0  9.145973e-15  378.395205  2.995334e-15  1.133420e-12  
 1  5.774606e-01  514.354737  1.294276e-01  6.657172e+01  
 2  4.700863e-01  489.302685  9.745898e-02  4.768694e+01  
 3  5.803060e-01  421.869227  1.171586e-01  4.942560e+01  
 4  5.929060e-01  555.417946  1.180323e-01  6.555724e+01  ,
    medium  t geo       C_inc
 0  Search  0  G1  378.395205
 1  Search  1  G1  514.354737
 2  Search  2  G1  489.302685
 3  Search  3  G1  421.869227
 4  Search  4  G1  555.417946,
    medium   campaign creative_family   objectiv

## 4) EM para desagregar C_inc a campañas (sin covariables): ecuaciones cerradas

In [4]:
import numpy as np
import pandas as pd

def em_mmm_to_mta_single_medium(sig_m, C_m, max_iter=200, tol=1e-8, verbose=True):
    df = sig_m.merge(C_m, on=["t","geo"], how="left")
    df["C_inc"] = df["C_inc"].fillna(0.0)

    campaigns = df["campaign"].unique().tolist()
    K = len(campaigns)
    camp_to_idx = {c:i for i,c in enumerate(campaigns)}

    k_idx = df["campaign"].map(camp_to_idx).to_numpy().astype(int)
    s = df["s_eff"].to_numpy().astype(float)

    tg = pd.factorize(list(zip(df["t"], df["geo"])))[0].astype(int)
    C = df["C_inc"].to_numpy().astype(float)

    n_groups = int(tg.max()) + 1

    lambda_hat = np.ones(K, dtype=float)
    history = {"rel_change": []}

    s_sum_k = np.bincount(k_idx, weights=s, minlength=K) + 1e-12

    for it in range(max_iter):
        num = lambda_hat[k_idx] * s
        den_g = np.bincount(tg, weights=num, minlength=n_groups) + 1e-12
        share = num / den_g[tg]
        zhat = C * share

        z_sum_k = np.bincount(k_idx, weights=zhat, minlength=K)
        lambda_new = z_sum_k / s_sum_k

        # normalize scale within medium (optional)
        lambda_new = lambda_new / (lambda_new.mean() + 1e-12)

        rel = float(np.linalg.norm(lambda_new - lambda_hat) / (np.linalg.norm(lambda_hat) + 1e-12))
        history["rel_change"].append(rel)

        if verbose and (it < 5 or it % 20 == 0):
            print(f"iter {it:3d} | rel_change={rel: .3e}")

        lambda_hat = lambda_new
        if rel < tol:
            break

    df_out = df[["campaign","t","geo","s_eff","C_inc"]].copy()
    df_out["zhat"] = zhat
    return pd.Series(lambda_hat, index=campaigns, name="lambda_hat"), df_out, history

results = []
lambda_hats = []
histories = {}

for m in media_names:
    sig_m = sig2.loc[sig2.medium==m, ["campaign","t","geo","s_eff"]].copy()
    C_m = C_df.loc[C_df.medium==m, ["t","geo","C_inc"]].copy()

    lam_hat, df_hat, hist = em_mmm_to_mta_single_medium(sig_m, C_m, verbose=False)
    df_hat["medium"] = m
    results.append(df_hat)

    lam_hat_df = lam_hat.reset_index().rename(columns={"index":"campaign"})
    lam_hat_df["medium"] = m
    lambda_hats.append(lam_hat_df)

    histories[m] = hist

zhat_df = pd.concat(results, ignore_index=True)
lambda_hat_df = pd.concat(lambda_hats, ignore_index=True)

lambda_hat_df.head(), zhat_df.head()


/tmp/ipykernel_13429/2429345959.py:15: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  tg = pd.factorize(list(zip(df["t"], df["geo"])))[0].astype(int)
/tmp/ipykernel_13429/2429345959.py:15: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  tg = pd.factorize(list(zip(df["t"], df["geo"])))[0].astype(int)


(    campaign  lambda_hat  medium
 0  Search_C1    1.061351  Search
 1  Search_C2    0.604222  Search
 2  Search_C3    0.718040  Search
 3  Search_C4    1.437445  Search
 4  Search_C5    1.313828  Search,
     campaign  t geo         s_eff       C_inc          zhat  medium
 0  Search_C1  0  G1  1.403196e-14  378.395205  2.161738e-12  Search
 1  Search_C1  1  G1  8.859533e-01  514.354737  1.066937e+02  Search
 2  Search_C1  2  G1  7.212172e-01  489.302685  7.821551e+01  Search
 3  Search_C1  3  G1  8.903188e-01  421.869227  7.988590e+01  Search
 4  Search_C1  4  G1  9.096500e-01  555.417946  1.059257e+02  Search)

## 5) Checks y evaluación (ground truth disponible en mock)

In [5]:
import numpy as np
import pandas as pd

chk = (zhat_df.groupby(["medium","t","geo"])["zhat"].sum()
       .reset_index()
       .merge(C_df, on=["medium","t","geo"], how="left"))
chk["gap"] = chk["zhat"] - chk["C_inc"]
print("max |gap| =", float(chk["gap"].abs().max()))

lam_true_df = campaign_df[["medium","campaign","lambda_true"]].copy()
lam_cmp = lam_true_df.merge(lambda_hat_df, on=["medium","campaign"], how="left")

corr_lam = lam_cmp.groupby("medium").apply(lambda d: np.corrcoef(d["lambda_true"], d["lambda_hat"])[0,1])
print("corr(lambda_true, lambda_hat) por medio:")
display(corr_lam)

lam_cmp.head()


max |gap| = 2.1509549696929753e-10
corr(lambda_true, lambda_hat) por medio:


/tmp/ipykernel_13429/1912274994.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  corr_lam = lam_cmp.groupby("medium").apply(lambda d: np.corrcoef(d["lambda_true"], d["lambda_hat"])[0,1])


medium
Search    0.716181
Social    0.484505
dtype: float64

,medium,campaign,lambda_true,lambda_hat
0,Search,Search_C1,0.651796,1.061351
1,Search,Search_C2,0.968844,0.604222
2,Search,Search_C3,0.478790,0.718040
3,Search,Search_C4,1.374762,1.437445
4,Search,Search_C5,1.654088,1.313828


In [6]:
import numpy as np
import pandas as pd

z_cmp = (sig2[["medium","campaign","t","geo","z_true"]]
         .merge(zhat_df[["medium","campaign","t","geo","zhat"]], on=["medium","campaign","t","geo"], how="left"))

for m in media_names:
    d = z_cmp.loc[z_cmp.medium==m]
    corr = np.corrcoef(d["z_true"], d["zhat"])[0,1]
    rmse = float(np.sqrt(np.mean((d["z_true"] - d["zhat"])**2)))
    print(m, "| corr(z_true, zhat) =", corr, "| rmse =", rmse)


Search | corr(z_true, zhat) = 0.7002968124473625 | rmse = 28.332376353147925
Social | corr(z_true, zhat) = 0.3973770228289043 | rmse = 68.3273734856492


## 6) Dónde entra adstock + saturación (resumen)


En este enfoque, adstock + saturación se usan para construir la exposición efectiva:

\[
w_{m,k,t,g} := s_{m,k,t,g} = \text{Hill}(\text{Adstock}(x_{m,k,t,g}))
\]

y luego el E-step reparte:

\[
\hat z_{m,k,t,g} = C_{m,t,g}^{inc} \cdot
\frac{\lambda_{m,k} w_{m,k,t,g}}
{\sum_{k'} \lambda_{m,k'} w_{m,k',t,g}}
\]

Esto respeta la estructura MMM y conserva masa: \(\sum_k \hat z = C\).


## 7) (Opcional) Extensión a features por campaña (lambda = exp(X beta))


Si quieres generalizar / regularizar:

\[
\lambda_{m,k} = \exp(X_{m,k} \beta_m)
\]

El M-step pasa a ser un GLM (Poisson/Gamma) con offset (igual que HCP–Brick).


## 8) Granularidad horaria (para más adelante)


Si el MMM está a nivel semanal/diario pero necesitas atribución horaria:

1) Ejecutas EM a nivel (t,g) y obtienes \(\hat z_{m,k,t,g}\)
2) Downscaling horario con perfil de delivery y conservación de masa:

\[
\hat z_{m,k,t,g,h} = \hat z_{m,k,t,g} \cdot
\frac{u_{m,k,t,g,h}}{\sum_h u_{m,k,t,g,h}}
\]

donde \(u\) puede ser impresiones horarias o un modelo de intensidad suavizado.
